Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model with ONNX Runtime on CPU

In this tutorial, you'll be introduced to how to load a Bert model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

This notebook is for CPU inference. For GPU inferenece, please look at another notebook [Inference PyTorch Bert Model with ONNX Runtime on GPU](PyTorch_Bert-Squad_OnnxRuntime_GPU.ipynb).

## 0. Prerequisites ##

If you have Jupyter Notebook, you may directly run this notebook. We will use pip to install or upgrade [PyTorch](https://pytorch.org/), [OnnxRuntime](https://microsoft.github.io/onnxruntime/) and other required packages.

Otherwise, you can setup a new environment. First, we install [AnaConda](https://www.anaconda.com/distribution/). Then open an AnaConda prompt window and run the following commands:

```console
conda create -n cpu_env python=3.6
conda activate cpu_env
conda install jupyter
jupyter notebook
```
The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

In [1]:
# Install or upgrade PyTorch 1.5.0 and OnnxRuntime 1.3.0 for CPU-only.
import sys
!{sys.executable} -m pip install --upgrade torch==1.5.0+cpu torchvision==0.6.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install --upgrade onnxruntime==1.3.0
!{sys.executable} -m pip install --upgrade onnxruntime-tools

# Install other packages used in this notebook.
!{sys.executable} -m pip install transformers==2.11.0
!{sys.executable} -m pip install wget netron

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: torch==1.5.0+cpu in d:\anaconda3\envs\cpu_env\lib\site-packages (1.5.0+cpu)
Requirement already up-to-date: torchvision==0.6.0+cpu in d:\anaconda3\envs\cpu_env\lib\site-packages (0.6.0+cpu)
Requirement already up-to-date: onnxruntime==1.3.0 in d:\anaconda3\envs\cpu_env\lib\site-packages (1.3.0)
Requirement already up-to-date: onnxruntime-tools in d:\anaconda3\envs\cpu_env\lib\site-packages (1.3.0.1007)


## 1. Load Pretrained Bert model ##

We begin by downloading the SQuAD data file and store them in the specified location.

In [2]:
import os

cache_dir = "./squad"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")

Specify some model configuration variables and constant.

In [3]:
# For fine tuned large model, the model name is "bert-large-uncased-whole-word-masking-finetuned-squad". Here we use bert-base for demo.
model_name_or_path = "bert-base-cased"
max_seq_length = 128
doc_stride = 128
max_query_length = 64

# Enable overwrite to export onnx model and download latest script each time when running this notebook.
enable_overwrite = True

# Total samples to inference. It shall be large enough to get stable latency measurement.
total_samples = 100

Start to load model from pretrained. This step could take a few minutes. 

In [4]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
# load some examples
from transformers.data.processors.squad import SquadV1Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features( 
            examples=examples[:total_samples], # convert just enough examples for this notebook
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

add example index and unique id: 100%|███████████████████████████████████████████| 100/100 [00:00<00:00, 100007.25it/s]


## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [5]:
output_dir = "./onnx"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)   
export_model_path = os.path.join(output_dir, 'bert-base-cased-squad.onnx')

import torch
device = torch.device("cpu")

# Get the first example data to run the model and export it to ONNX
data = dataset[0]
inputs = {
    'input_ids':      data[0].to(device).reshape(1, max_seq_length),
    'attention_mask': data[1].to(device).reshape(1, max_seq_length),
    'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
}

# Set model to inference mode, which is required before exporting the model because some operators behave differently in 
# inference and training mode.
model.eval()
model.to(device)

if enable_overwrite or not os.path.exists(export_model_path):
    with torch.no_grad():
        symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
        torch.onnx.export(model,                                            # model being run
                          args=tuple(inputs.values()),                      # model input (or a tuple for multiple inputs)
                          f=export_model_path,                              # where to save the model (can be a file or file-like object)
                          opset_version=11,                                 # the ONNX version to export the model to
                          do_constant_folding=True,                         # whether to execute constant folding for optimization
                          input_names=['input_ids',                         # the model's input names
                                       'input_mask', 
                                       'segment_ids'],
                          output_names=['start', 'end'],                    # the model's output names
                          dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                        'input_mask' : symbolic_names,
                                        'segment_ids' : symbolic_names,
                                        'start' : symbolic_names,
                                        'end' : symbolic_names})
        print("Model exported at ", export_model_path)

Model exported at  ./onnx\bert-base-cased-squad.onnx


## 3. PyTorch Inference ##
Use PyTorch to evaluate an example input for comparison purpose.

In [6]:
import time

# Measure the latency. It is not accurate using Jupyter Notebook, it is recommended to use standalone python script.
latency = []
with torch.no_grad():
    for i in range(total_samples):
        data = dataset[i]
        inputs = {
            'input_ids':      data[0].to(device).reshape(1, max_seq_length),
            'attention_mask': data[1].to(device).reshape(1, max_seq_length),
            'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
        }
        start = time.time()
        outputs = model(**inputs)
        latency.append(time.time() - start)
print("PyTorch {} Inference time = {} ms".format(device.type, format(sum(latency) * 1000 / len(latency), '.2f')))

PyTorch cpu Inference time = 198.99 ms


## 4. Inference ONNX Model with ONNX Runtime ##

### OpenMP Environment Variable

OpenMP environment variables are very important for CPU inference of Bert model. It has large performance impact on Bert model so you might need set it carefully according to [Performance Test Tool](#Performance-Test-Tool) result in later part of this notebook.

Setting environment variables shall be done before importing onnxruntime. Otherwise, they might not take effect.

In [7]:
import psutil

# You may change the settings in this cell according to Performance Test Tool result.
os.environ["OMP_NUM_THREADS"] = str(psutil.cpu_count(logical=True))
os.environ["OMP_WAIT_POLICY"] = 'ACTIVE'

Now we are ready to inference the model with ONNX Runtime. Here we can see that OnnxRuntime has better performance than PyTorch. 

It is better to use standalone python script like [Performance Test tool](#Performance-Test-tool) to get accurate performance results.

In [8]:
import onnxruntime
import numpy

# Print warning if user uses onnxruntime-gpu instead of onnxruntime package.
if 'CUDAExecutionProvider' in onnxruntime.get_available_providers():
    print("warning: onnxruntime-gpu is not built with OpenMP. You might try onnxruntime package to test CPU inference.")

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
# Note that this will increase session creation time, so it is for debugging only.
sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_cpu.onnx")

# intra_op_num_threads is needed for OnnxRuntime 1.2.0.
# For OnnxRuntime 1.3.0 or later, this does not have effect unless you are using onnxruntime-gpu package.
sess_options.intra_op_num_threads=1


# Specify providers when you use onnxruntime-gpu for CPU inference.
session = onnxruntime.InferenceSession(export_model_path, sess_options, providers=['CPUExecutionProvider'])

latency = []
for i in range(total_samples):
    data = dataset[i]
    # TODO: use IO Binding (see https://github.com/microsoft/onnxruntime/pull/4206) to improve performance.
    ort_inputs = {
        'input_ids':  data[0].cpu().reshape(1, max_seq_length).numpy(),
        'input_mask': data[1].cpu().reshape(1, max_seq_length).numpy(),
        'segment_ids': data[2].cpu().reshape(1, max_seq_length).numpy()
    }
    start = time.time()
    ort_outputs = session.run(None, ort_inputs)
    latency.append(time.time() - start)
print("OnnxRuntime cpu Inference time = {} ms".format(format(sum(latency) * 1000 / len(latency), '.2f')))

OnnxRuntime cpu Inference time = 176.96 ms


In [9]:
print("***** Verifying correctness *****")
for i in range(2):
    print('PyTorch and ONNX Runtime output {} are close:'.format(i), numpy.allclose(ort_outputs[i], outputs[i].cpu(), rtol=1e-05, atol=1e-04))

***** Verifying correctness *****
PyTorch and ONNX Runtime output 0 are close: True
PyTorch and ONNX Runtime output 1 are close: True


## 5. Offline Optimization Script and Test Tools

It is recommended to try [OnnxRuntime Transformer Model Optimization Tool](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/transformers) on the exported ONNX models. It could help verify whether the model can be fully optimized, and get performance test results.

#### Transformer Optimizer

Although OnnxRuntime could optimize Bert model exported by PyTorch. Sometime, model cannot be fully optimized due to different reasons:
* A new subgraph pattern is generated by new version of export tool, and the pattern is not covered by older version of OnnxRuntime. 
* The exported model uses dynamic axis and this makes it harder for shape inference of the graph. That blocks some optimization to be applied.
* Some optimization is better to be done offline. Like change input tensor type from int64 to int32 to avoid extra Cast nodes, or convert model to float16 to achieve better performance in V100 or T4 GPU.

We have python script **optimizer.py**, which is more flexible in graph pattern matching and model conversion (like float32 to float16). You can also use it to verify whether a Bert model is fully optimized.

In this example, we can see that it introduces optimization that is not provided by onnxruntime: SkipLayerNormalization and bias fusion, which is not fused in OnnxRuntime due to shape inference as mentioned.

It will also tell whether the model is fully optimized or not. If not, that means you might need change the script to fuse some new pattern of subgraph.

Example Usage:
```
from onnxruntime_tools import optimizer
optimized_model = optimizer.optimize_model(export_model_path, model_type='bert', num_heads=12, hidden_size=768)
optimized_model.save_model_to_file(optimized_model_path)
```

You can also use optimizer_cli like the following:

In [10]:
optimized_model_path = './onnx/bert-base-cased-squad_opt_cpu.onnx'

!{sys.executable} -m onnxruntime_tools.optimizer_cli --input $export_model_path --output $optimized_model_path --model_type bert --num_heads 12 --hidden_size 768

optimize_by_onnxruntime: Save optimized model by onnxruntime to ./onnx\bert-base-cased-squad_o1_cpu.onnx
               apply: Fused LayerNormalization count: 25
               apply: Fused LayerNormalization count: 0
               apply: Fused Gelu count: 12
               apply: Fused FastGelu count: 0
               apply: Fused Reshape count: 0
               apply: Fused SkipLayerNormalization count: 25
               apply: Fused Attention count: 12
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 5 nodes are removed
               apply: Fused EmbedLayerNormalization(with mask) count: 1
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 12 nodes are removed
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 0 nodes are removed
               apply: Fused FastGelu(add bias) count: 0
               apply: Fused BiasGelu count: 12
               apply: Fused SkipLayerNormalization(add bias) count: 24
            optimize: opset verion: 11
  save_model_

#### Optimized Graph
When you can open the optimized model using Netron to visualize, the graph is like the following:
<img src='images/optimized_bert_gpu.png'>

For CPU, optimized graph is slightly different: FastGelu is replaced by BiasGelu.

In [11]:
import netron

# Change it to False to skip viewing the optimized model in browser.
enable_netron = True
if enable_netron:
    # If you encounter error "access a socket in a way forbidden by its access permissions", install Netron as standalone application instead.
    netron.start(optimized_model_path)

Serving './onnx/bert-base-cased-squad_opt_cpu.onnx' at http://localhost:8080


#### Model Results Comparison Tool

If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare results from both the original and optimized models. If outputs are all close, it is safe to use the optimized model.

Example of verifying models:

In [12]:
!{sys.executable} -m onnxruntime_tools.transformers.compare_bert_results --baseline_model $export_model_path --optimized_model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100

100% passed for 100 random inputs given thresholds (rtol=0.001, atol=0.0001).
maximum absolute difference=3.46451997756958e-06
maximum relative difference=0.03302651643753052


#### Performance Test Tool

This tool measures performance of BERT model inference using OnnxRuntime Python API.

The following command will create 100 samples of batch_size 1 and sequence length 128 to run inference, then calculate performance numbers like average latency and throughput etc. You can increase number of samples (recommended 1000) to get more stable result.

In [16]:
!{sys.executable} -m onnxruntime_tools.transformers.bert_perf_test --model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100 --test_times 1 --intra_op_num_threads 1 --inclusive --all

Running test: model=bert-base-cased-squad_opt_cpu.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=1,OMP_NUM_THREADS=,OMP_WAIT_POLICY=,batch_size=1,sequence_length=128,test_cases=100,test_times=1,contiguous=False,use_gpu=False,warmup=True
Average latency = 102.17 ms, Throughput = 9.79 QPS
Running test: model=bert-base-cased-squad_opt_cpu.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=1,OMP_NUM_THREADS=,OMP_WAIT_POLICY=PASSIVE,batch_size=1,sequence_length=128,test_cases=100,test_times=1,contiguous=False,use_gpu=False,warmup=True
Average latency = 91.39 ms, Throughput = 10.94 QPS
Running test: model=bert-base-cased-squad_opt_cpu.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=1,OMP_NUM_THREADS=,OMP_WAIT_POLICY=ACTIVE,batch_size=1,sequence_length=128,test_cases=100,test_times=1,contiguous=False,use_gpu=False,warmup=True
Average latency = 105.09 ms, Throughput = 9.52 QPS
Running test: model=bert-base-cased-squad_opt_cpu.onnx,graph_optimization_leve

Let's load the summary file and take a look.

In [17]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print(latest_result_file)

# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu', 'warmup']
# Hide some latency percentile columns to fit screen width.
columns_to_remove.extend(['Latency_P50', 'Latency_P95'])
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

./onnx\perf_results_CPU_B1_S128_20200612-115010.txt
The best setting is: use openmp; NO contiguous array


,Latency(ms),Latency_P75,Latency_P90,Latency_P99,Throughput(QPS),intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous
0,84.73,87.19,91.51,95.46,11.80,1,12,PASSIVE,False
1,91.39,94.92,101.16,119.06,10.94,1,,PASSIVE,False
2,100.54,101.63,103.12,109.20,9.95,1,6,,False
3,101.14,102.05,106.15,112.17,9.89,1,6,ACTIVE,False
4,101.53,102.30,105.58,113.53,9.85,1,12,ACTIVE,False
5,101.56,102.54,104.08,106.72,9.85,1,12,,False
6,102.17,104.58,106.50,111.18,9.79,1,,,False
7,105.09,107.26,112.70,123.89,9.52,1,,ACTIVE,False
8,111.82,113.28,116.62,119.12,8.94,1,6,PASSIVE,False
9,289.82,294.33,300.76,333.44,3.45,1,1,ACTIVE,False


## 6. Additional Info

Note that running Jupyter Notebook has slight impact on performance result since Jupyter Notebook is using system resources like CPU and memory etc. It is recommended to close Jupyter Notebook and other applications, then run the performance test tool in a console to get more accurate performance numbers.

We have a [benchmark script](https://github.com/microsoft/onnxruntime/blob/master/onnxruntime/python/tools/transformers/run_benchmark.sh). It is recommended to use it compare inference speed of OnnxRuntime with PyTorch.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/master/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. The machine has GPU but not used in CPU inference.
You might get slower or faster result based on your hardware.

In [18]:
!{sys.executable} -m onnxruntime_tools.transformers.machine_info --silent

{
  "gpu": {
    "driver_version": "442.23",
    "devices": [
      {
        "memory_total": 8589934592,
        "memory_available": 8480882688,
        "name": "GeForce GTX 1070"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz",
    "cores": 6,
    "logical_cores": 12,
    "hz": "3.1920 GHz",
    "l2_cache": "1536 KB",
    "l3_cache": "12288 KB",
    "processor": "Intel64 Family 6 Model 158 Stepping 10, GenuineIntel"
  },
  "memory": {
    "total": 16971259904,
    "available": 6604914688
  },
  "python": "3.6.10.final.0 (64 bit)",
  "os": "Windows-10-10.0.18362-SP0",
  "onnxruntime": {
    "version": "1.3.0",
    "support_gpu": false
  },
  "pytorch": {
    "version": "1.5.0+cpu",
    "support_gpu": false
  },
  "tensorflow": {
    "version": "2.1.0",
    "git_version": "v2.1.0-rc2-17-ge5bf8de410",
    "support_gpu": true
  }
}


2020-06-12 11:50:22.559054: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
